In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -U ensemble-boxes


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np
from ensemble_boxes import *
from glob import glob
import copy
from tqdm import tqdm
import shutil
import zipfile

In [4]:
height_dict = pd.read_csv('/content/drive/MyDrive/project/vinBigData/test.csv').to_dict('records')
fnl_dict ={}
for ix,i in enumerate(height_dict):
    fnl_dict[i['image_id']] = [i['width'],i['height'],i['width'],i['height']]

In [5]:
#합칠 파일 
subs = [

    pd.read_csv('/content/mmdedtection_faster_aws_2_1_30.csv'),
    pd.read_csv('/content/mmdedtection_retina_aws_1024C_3_30.csv'),
    pd.read_csv('/content/mmdedtection_retina_aws_1_30.csv'),
    pd.read_csv('/content/mmdedtection_yolof_aws_1_04_30.csv'),
    pd.read_csv('/content/mmdedtection_yolof_aws_2_06_1024B_30.csv'),
    pd.read_csv('/content/mmdedtection_yolox_aws_2_1024C_40.csv'),
    pd.read_csv('/content/yolox_1024_adamw_A.csv'),


       ]
pred_2cls = pd.read_csv('/content/2-cls test pred.csv')

In [6]:
def submission_decoder(df:pd.DataFrame) -> np.ndarray:
    info = df.values
    df_lst = []
    for i in info:
        pre_lst = i[1].split(' ')
        if(pre_lst[-1] == ''):
            pre_lst = pre_lst[:-1]
        for j in range(0,len(pre_lst),6):
            df_lst.append([i[0],int(pre_lst[j]),float(pre_lst[j+1]),int(float(pre_lst[j+2])),int(float(pre_lst[j+3])),\
                           int(float(pre_lst[j+4])),int(float(pre_lst[j+5])),fnl_dict.get(i[0])[0],fnl_dict.get(i[0])[1]])
    return pd.DataFrame(df_lst,columns = ['image_id','class_id','score','x_min','y_min','x_max','y_max','width','height'])

In [7]:
subs = [submission_decoder(subs[i]) for i in range(len(subs))]

In [8]:
boxes_dict = {}
scores_dict = {}
labels_dict = {}
whwh_dict = {}

for i in tqdm(subs[0].image_id.unique()):
    if not i in boxes_dict.keys():
        boxes_dict[i] = []
        scores_dict[i] = []
        labels_dict[i] = []
        whwh_dict[i] = []

    size_ratio = fnl_dict.get(i)
    whwh_dict[i].append(size_ratio) 
    tmp_df = [subs[x][subs[x]['image_id']==i] for x in range(len(subs))]
    
    for x in range(len(tmp_df)):
        boxes_dict[i].append(((tmp_df[x][['x_min','y_min','x_max','y_max']].values)/size_ratio).tolist())
        scores_dict[i].append(tmp_df[x]['score'].values.tolist())
        labels_dict[i].append(tmp_df[x]['class_id'].values.tolist())

100%|██████████| 3000/3000 [00:45<00:00, 65.32it/s]


In [9]:
weights = [1]*2#n = 1
#n=넣는 파일 갯수 -1(*n)
# 현재 2개를 합지기 때문에1
weights += [3]
weights1 = [3,2,4,5]
iou_thr = 0.25
skip_box_thr = 0.0
sigma = 0.1

fnl = {}

for i in tqdm(boxes_dict.keys()):
    
    
    boxes, scores, labels = weighted_boxes_fusion(boxes_dict[i], scores_dict[i], labels_dict[i],\
                                                  weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    
    
    if not i in fnl.keys():
        fnl[i] = {'boxes':[],'scores':[],'labels':[]}
        
    fnl[i]['boxes'] = boxes*whwh_dict[i]
    fnl[i]['scores'] = scores
    fnl[i]['labels'] = labels

  3%|▎         | 78/3000 [00:00<00:07, 385.05it/s]

  5%|▌         | 155/3000 [00:00<00:07, 366.57it/s]

  8%|▊         | 235/3000 [00:00<00:07, 378.22it/s]

 11%|█         | 316/3000 [00:00<00:06, 391.05it/s]

 13%|█▎        | 398/3000 [00:01<00:06, 388.68it/s]

...

 91%|█████████ | 2722/3000 [00:06<00:00, 406.56it/s]

 94%|█████████▎| 2808/3000 [00:07<00:00, 410.04it/s]

 95%|█████████▌| 2850/3000 [00:07<00:00, 365.42it/s]

 97%|█████████▋| 2922/3000 [00:07<00:00, 320.39it/s]

100%|██████████| 3000/3000 [00:07<00:00, 387.77it/s]

In [10]:
pd_form = []
for i in fnl.keys():
    b = fnl[i]
    for j in range(len(b['boxes'])):
        try:
          pd_form.append([i,int(b['labels'][j]),round(b['scores'][j],2),\
                          int(b['boxes'][j][0]),int(b['boxes'][j][1]),\
                          int(b['boxes'][j][2]),int(b['boxes'][j][3])])
        except:
          pd_form.append([i,14 ,1.0, 0 ,0 ,1 ,1])
        
final_df = pd.DataFrame(pd_form,columns = ['image_id','class_id','score','x_min','y_min','x_max','y_max'])
final_df = final_df.drop_duplicates(keep = 'first')

In [11]:
def submission_encoder(df:pd.DataFrame) -> np.ndarray:
    dct = {}
    for i in tqdm(df['image_id'].unique()):
        if not i in dct.keys():
            dct[i] = []
        tmp = df[df['image_id'] == i].values
        for j in tmp:
            dct[i].append(int(j[1]))
            dct[i].append(float(j[2]))
            dct[i].append(int(j[3]))
            dct[i].append(int(j[4]))
            dct[i].append(int(j[5]))
            dct[i].append(int(j[6]))
        
        dct[i] = map(str,dct[i])
        dct[i] = ' '.join(dct[i])
    dct = [[k, v] for k, v in dct.items()]
    return pd.DataFrame(dct,columns = ['image_id','PredictionString']).reset_index(drop = True)

df = submission_encoder(final_df)
df.to_csv('Fold5Yolo.csv', index=False)

100%|██████████| 3000/3000 [00:08<00:00, 341.63it/s]


In [12]:
NORMAL = "14 1 0 0 1 1"
low_threshold = 0.00
high_threshold = 0.99
pred_det_df = df  # You can load from another submission.csv here too.
n_normal_before = len(pred_det_df.query("PredictionString == @NORMAL"))
merged_df = pd.merge(pred_det_df, pred_2cls, on="image_id", how="left")

if "target" in merged_df.columns:
    merged_df["class0"] = 1 - merged_df["target"]

c0, c1, c2 = 0, 0, 0
for i in range(len(merged_df)):
    p0 = merged_df.loc[i, "class0"]
    if p0 < low_threshold:
        # Keep, do nothing.
        c0 += 1
    elif low_threshold <= p0 and p0 < high_threshold:
        # Add, keep "det" preds and add normal pred.
        if ' 14 ' not in merged_df.loc[i, "PredictionString"]:
            merged_df.loc[i, "PredictionString"] += f" 14 {p0} 0 0 1 1"
            
        c1 += 1
    else:
        # Replace, remove all "det" preds.
        merged_df.loc[i, "PredictionString"] = NORMAL
        c2 += 1

n_normal_after = len(merged_df.query("PredictionString == @NORMAL"))
print(
    f"n_normal: {n_normal_before} -> {n_normal_after} with threshold {low_threshold} & {high_threshold}"
)
print(f"Keep {c0} Add {c1} Replace {c2}")
submission_filepath = str("submission.csv")
submission_df = merged_df[["image_id", "PredictionString"]]
submission_df.to_csv(submission_filepath, index=False)
print(f"Saved to {submission_filepath}")

n_normal: 0 -> 292 with threshold 0.0 & 0.99
Keep 0 Add 2708 Replace 292
Saved to submission.csv
